# Canvas Animation Extractor for Wuthering Waves - Google Colab Version

This notebook extracts animated canvas content from web pages and converts them to high-quality MP4 videos.

**Features:**
- Web scraping with Selenium and Chrome
- Canvas animation frame capture
- High-resolution image processing with PIL
- Video conversion with FFmpeg
- Optimized for Google Colab environment

**Target Website:** Wuthering Waves Event Page

## Step 1: Install Required Dependencies

First, we'll install all the necessary packages for web scraping, image processing, and video conversion.

In [ ]:
# Install required packages
!apt-get update
!apt-get install -y chromium-driver ffmpeg

# Install Python packages
!pip install selenium pillow numpy opencv-python

# Install additional dependencies for video processing
!pip install imageio imageio-ffmpeg

print('All dependencies installed successfully!')

## Step 2: Import Required Libraries

Import all the necessary libraries for web scraping, image processing, and video creation.

In [ ]:
import os
import time
import base64
import json
from datetime import datetime
from pathlib import Path

# Web scraping and browser automation
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# Image processing
from PIL import Image
import numpy as np
import cv2

# Video creation
import imageio

print('All libraries imported successfully!')
print(f'Current time: {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}')

## Step 3: Configuration Settings

Set up the configuration parameters for canvas capture and video creation.

In [ ]:
# Configuration settings
CONFIG = {
    'url': 'https://wutheringwaves-event1.kurogames-global.com/?packageId=A1730&language=en&isInternalBrowser=0&platform=PC',
    'max_frames': 600,
    'capture_fps': 30,
    'output_fps': 30,
    'viewport_width': 3840,
    'viewport_height': 2160,
    'wait_time': 10,
    'frame_interval': 1.0/30,
    'output_dir': 'canvas_frames',
    'video_name': 'wuthering_waves_animation.mp4'
}

# Create output directory
os.makedirs(CONFIG['output_dir'], exist_ok=True)

print('Configuration loaded:')
for key, value in CONFIG.items():
    print(f'   {key}: {value}')

## Step 4: Browser Setup Functions

Create functions to set up Chrome browser with optimal settings for canvas capture.

In [ ]:
def setup_chrome_driver():
    chrome_options = Options()

    # Essential Chrome options for Colab environment
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    chrome_options.add_argument('--disable-gpu')
    chrome_options.add_argument('--disable-features=VizDisplayCompositor')
    chrome_options.add_argument('--disable-web-security')
    chrome_options.add_argument('--allow-running-insecure-content')
    chrome_options.add_argument('--disable-blink-features=AutomationControlled')

    # Set window size for high resolution capture
    chrome_options.add_argument(f'--window-size={CONFIG["viewport_width"]},{CONFIG["viewport_height"]}')

    # Performance optimizations
    chrome_options.add_argument('--disable-extensions')
    chrome_options.add_argument('--disable-plugins')

    try:
        driver = webdriver.Chrome(options=chrome_options)
        driver.implicitly_wait(10)
        driver.set_page_load_timeout(60)
        print('Chrome driver setup successfully!')
        return driver
    except Exception as e:
        print(f'Failed to setup Chrome driver: {e}')
        return None

print('Browser setup function ready!')

## Step 5: Canvas Capture Functions

Create functions to detect and capture canvas animations from the web page.

In [ ]:
def detect_canvas_elements(driver):
    try:
        canvas_elements = WebDriverWait(driver, 20).until(
            EC.presence_of_all_elements_located((By.TAG_NAME, 'canvas'))
        )

        canvas_info = []
        for i, canvas in enumerate(canvas_elements):
            try:
                info = driver.execute_script("""
                    var canvas = arguments[0];
                    return {
                        width: canvas.width,
                        height: canvas.height,
                        offsetWidth: canvas.offsetWidth,
                        offsetHeight: canvas.offsetHeight,
                        id: canvas.id,
                        className: canvas.className
                    };
                """, canvas)
                info['index'] = i
                info['element'] = canvas
                canvas_info.append(info)
            except Exception as e:
                print(f'Error getting info for canvas {i}: {e}')

        return canvas_info
    except Exception as e:
        print(f'Error detecting canvas elements: {e}')
        return []

def capture_canvas_frame(driver, canvas_element):
    try:
        canvas_data = driver.execute_script("""
            var canvas = arguments[0];
            try {
                return canvas.toDataURL('image/png');
            } catch (e) {
                return null;
            }
        """, canvas_element)

        if canvas_data and canvas_data.startswith('data:image/png;base64,'):
            return canvas_data
        else:
            return None
    except Exception as e:
        return None

def save_frame(canvas_data, frame_number, output_dir):
    try:
        base64_data = canvas_data.split(',')[1]
        image_data = base64.b64decode(base64_data)

        filename = f'frame_{frame_number:06d}.png'
        filepath = os.path.join(output_dir, filename)

        with open(filepath, 'wb') as f:
            f.write(image_data)

        return filepath
    except Exception as e:
        print(f'Error saving frame {frame_number}: {e}')
        return None

print('Canvas capture functions ready!')

## Step 6: Main Canvas Animation Capture

The main function that orchestrates the entire canvas capture process.

In [ ]:
def capture_canvas_animation():
    print('Starting canvas animation capture...')
    print(f'Target URL: {CONFIG["url"]}')

    driver = setup_chrome_driver()
    if not driver:
        return False, []

    try:
        print('Loading website...')
        driver.get(CONFIG['url'])

        print(f'Waiting {CONFIG["wait_time"]} seconds for page load...')
        time.sleep(CONFIG['wait_time'])

        print('Detecting canvas elements...')
        canvas_info = detect_canvas_elements(driver)

        if not canvas_info:
            print('No canvas elements found!')
            return False, []

        print(f'Found {len(canvas_info)} canvas elements:')
        for info in canvas_info:
            print(f'   Canvas {info["index"]}: {info["width"]}x{info["height"]}')

        target_canvas = max(canvas_info, key=lambda x: x['width'] * x['height'])
        print(f'Selected canvas {target_canvas["index"]} for capture')

        print('Starting frame capture...')
        captured_frames = []
        frame_count = 0
        consecutive_failures = 0

        start_time = time.time()

        while frame_count < CONFIG['max_frames'] and consecutive_failures < 10:
            canvas_data = capture_canvas_frame(driver, target_canvas['element'])

            if canvas_data:
                filepath = save_frame(canvas_data, frame_count, CONFIG['output_dir'])
                if filepath:
                    captured_frames.append(filepath)
                    consecutive_failures = 0

                    if frame_count % 30 == 0:
                        elapsed = time.time() - start_time
                        print(f'Captured {frame_count + 1} frames ({elapsed:.1f}s elapsed)')
                else:
                    consecutive_failures += 1
            else:
                consecutive_failures += 1

            frame_count += 1
            time.sleep(CONFIG['frame_interval'])

        total_time = time.time() - start_time
        print(f'Capture completed! Total frames: {len(captured_frames)}')
        print(f'Total time: {total_time:.1f}s')

        return len(captured_frames) > 0, captured_frames

    except Exception as e:
        print(f'Error during capture: {e}')
        return False, []

    finally:
        driver.quit()
        print('Browser closed')

print('Main capture function ready!')

## Step 7: Video Conversion Functions

Convert the captured frames into a high-quality MP4 video.

In [ ]:
def create_video_from_frames(frame_files, output_path, fps=30):
    if not frame_files:
        print('No frames to convert!')
        return False

    try:
        print(f'Creating video from {len(frame_files)} frames...')
        print(f'Output: {output_path}')
        print(f'FPS: {fps}')

        frame_files.sort()

        first_frame = cv2.imread(frame_files[0])
        if first_frame is None:
            print('Could not read first frame!')
            return False

        height, width, layers = first_frame.shape
        print(f'Video dimensions: {width}x{height}')

        with imageio.get_writer(output_path, fps=fps, quality=9, codec='libx264') as writer:
            for i, frame_file in enumerate(frame_files):
                frame = cv2.imread(frame_file)
                if frame is not None:
                    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                    writer.append_data(frame_rgb)

                    if i % 30 == 0:
                        progress = (i + 1) / len(frame_files) * 100
                        print(f'Progress: {progress:.1f}%')

        print(f'Video created successfully: {output_path}')
        file_size = os.path.getsize(output_path) / (1024 * 1024)
        print(f'File size: {file_size:.1f} MB')

        return True

    except Exception as e:
        print(f'Error creating video: {e}')
        return False

print('Video conversion functions ready!')

## Step 8: Execute Canvas Animation Capture

Run the complete canvas animation capture and video creation process.

In [ ]:
# Main execution
print('=' * 60)
print('CANVAS ANIMATION EXTRACTOR FOR WUTHERING WAVES')
print('=' * 60)

# Step 1: Capture canvas animation
success, captured_frames = capture_canvas_animation()

if success and captured_frames:
    print(f'Successfully captured {len(captured_frames)} frames!')

    # Step 2: Create video from frames
    print('Creating video...')
    video_output = CONFIG['video_name']

    video_success = create_video_from_frames(
        captured_frames,
        video_output,
        fps=CONFIG['output_fps']
    )

    if video_success:
        print(f'SUCCESS! Video created: {video_output}')
        print('You can download the video file from the Colab file browser!')
    else:
        print('Video creation failed!')
else:
    print('Canvas capture failed!')

print('Process completed!')